In [16]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

import re
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import text

In [17]:
# Example URLs

url = "https://www.eversports.de/sw/poda-studio"
# 'https://www.eversports.de/sw/aerialflow'
# 'https://www.eversports.de/s/yoga-and-pole-art-by-selina'
# 'https://www.eversports.de/sw/schoenheitstanz-studio'
# 'https://www.eversports.de/sw/poda-studio'

In [18]:
# Send a GET request to the webpage

response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')
response

<Response [200]>

Container Segment 1 Dates

In [19]:
# Container Workshops
list_1 = soup.find_all('div', class_="discover-results marketplace-tile-container")

In [20]:
# Assuming you have already fetched the HTML content and parsed it using BeautifulSoup
list_1 = soup.find_all('div', class_='marketplace-tile__date')

workshops_dates = []

# Loop through each div element and extract its text content
for item in list_1:
    workshops_dates.append(item.text)
    print(item.text)


Mai07Sonntag
Mai07Sonntag
Mai07Sonntag
Mai07Sonntag
Mai14Sonntag
Juni04Sonntag
Juni10Samstag
Juni10Samstag
Juli02Sonntag
Aug06Sonntag
Sep03Sonntag
Okt01Sonntag
Nov05Sonntag
Dez03Sonntag


Container Segment 2 Name

In [21]:
# Create list of names of the workshops
workshop_names = []

workshops = soup.find_all('div', class_="discover-results marketplace-tile-container")

for item in workshops:
    h4_tags = item.find_all('h4')
    for tag in h4_tags:
        workshop_names.append(tag.text)

workshop_names

['Circus Kids Luftakrobatik Workshop',
 'Pole Dance Probestunde *Poda Birthday Special* Free',
 'Pole Fitness *Poda Birthday Special* Free',
 'Twerk & Chair *Poda Birthday Special* Free',
 'Hard Style Exotic Pole Workshop',
 'Circus Kids Luftakrobatik Workshop',
 '*Fortsetzung folgt...* Lady Marmalade Chair Dance Workshop',
 '*Fortsetzung folgt...* Azzlack Stereotyp 2 - Exotic Pole Workshop',
 'Circus Kids Luftakrobatik Workshop',
 'Circus Kids Luftakrobatik Workshop',
 'Circus Kids Luftakrobatik Workshop',
 'Circus Kids Luftakrobatik Workshop',
 'Circus Kids Luftakrobatik Workshop',
 'Circus Kids Luftakrobatik Workshop']

In [22]:
name_soup= soup.find_all("div", class_ = "container discover-container")
name_soup

for item in name_soup:
    h1_tag = item.find_all("h1")
    for h1 in h1_tag:
        polestudio_name = h1.text

polestudio_name_cl = polestudio_name.split(" ")[0]     
polestudio_name_cl   

'poda'

Container Segment 3 Price Workshops

In [23]:
# Extract all div elements with class "marketplace-tile__price"
price_elements = soup.find_all('div', class_='marketplace-tile__price')

# Extract the text content of each price element
workshop_prices = [price.text.strip() for price in price_elements]

# Print the list of workshop prices
print(workshop_prices)


['ab 35,00 €', 'Ausgebucht', 'Ausgebucht', 'Ausgebucht', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €', 'ab 35,00 €']


Weblinks

In [24]:
# find all links inside the container and create a list

links = [a['href'] for a in soup.find('div', class_='discover-results marketplace-tile-container').find_all('a')]

workshop_page_eversports = []
base_url = "https://www.eversports.de"

workshop_page_eversports = [base_url + link for link in links]
workshop_page_eversports


['https://www.eversports.de/e/workshop/x1w0Mq3Y7',
 'https://www.eversports.de/e/workshop/xFwPArnAH',
 'https://www.eversports.de/e/workshop/Sh6fqaIks',
 'https://www.eversports.de/e/workshop/feME-_7tG',
 'https://www.eversports.de/e/workshop/dCR2PVd05',
 'https://www.eversports.de/e/workshop/nb8_WV2wH',
 'https://www.eversports.de/e/workshop/cDPxswQQp',
 'https://www.eversports.de/e/workshop/xo_EQbOQn',
 'https://www.eversports.de/e/workshop/HGkpQ9uAJ',
 'https://www.eversports.de/e/workshop/Bz5neTPTa',
 'https://www.eversports.de/e/workshop/3zdELX6G_',
 'https://www.eversports.de/e/workshop/tUZ-pbNF0',
 'https://www.eversports.de/e/workshop/5u5RfrpDC',
 'https://www.eversports.de/e/workshop/RPiNq9mm6']

In [26]:
# create a dictionary from the given lists
poleworkshop_list_df = {
    'Workshop Name': workshop_names,
    'PoleStudio': polestudio_name_cl,
    # 'Workshop Datum': workshops_dates,
    # 'Workshop Preis': workshop_prices,
    'Workshop Seite Eversports': workshop_page_eversports,
    'Eversports Page': url
    
}

# create pandas dataframe
poleworkshop_list_df = pd.DataFrame.from_dict(poleworkshop_list_df)

# Add a new column to the DataFrame with the current date and time
poleworkshop_list_df['Version'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# display dataframe
poleworkshop_list_df

,Workshop Name,PoleStudio,Workshop Seite Eversports,Eversports Page,Version
0,Circus Kids Luftakrobatik Workshop,poda,https://www.eversports.de/e/workshop/x1w0Mq3Y7,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
1,Pole Dance Probestunde *Poda Birthday Special*...,poda,https://www.eversports.de/e/workshop/xFwPArnAH,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
2,Pole Fitness *Poda Birthday Special* Free,poda,https://www.eversports.de/e/workshop/Sh6fqaIks,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
3,Twerk & Chair *Poda Birthday Special* Free,poda,https://www.eversports.de/e/workshop/feME-_7tG,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
4,Hard Style Exotic Pole Workshop,poda,https://www.eversports.de/e/workshop/dCR2PVd05,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
5,Circus Kids Luftakrobatik Workshop,poda,https://www.eversports.de/e/workshop/nb8_WV2wH,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
6,*Fortsetzung folgt...* Lady Marmalade Chair Da...,poda,https://www.eversports.de/e/workshop/cDPxswQQp,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
7,*Fortsetzung folgt...* Azzlack Stereotyp 2 - E...,poda,https://www.eversports.de/e/workshop/xo_EQbOQn,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
8,Circus Kids Luftakrobatik Workshop,poda,https://www.eversports.de/e/workshop/HGkpQ9uAJ,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28
9,Circus Kids Luftakrobatik Workshop,poda,https://www.eversports.de/e/workshop/Bz5neTPTa,https://www.eversports.de/sw/poda-studio,2023-05-05 15:57:28


In [27]:
# create the directory if it doesn't exist
# if not os.path.exists("Workshop_List_CSV"):
#     os.makedirs("Workshop_List_CSV")

# save the CSV file in the "CSV" directory
poleworkshop_list_df.to_csv(f"{polestudio_name_cl}.csv", index=False)

SQL Database

In [28]:
# SQL Setup

# host = 'Your information!'
# schema = 'hallofpole'
# user = 'Your information!'
# password = 'Your information!'
# port = Your information!

# # create connection string with the database name included
# con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# # create sqlalchemy engine
# engine = create_engine(con)

In [29]:
# # Write the DataFrame to the SQL database
# poleworkshop_list_df.to_sql(name='workshop_list', con=con, if_exists='replace', index=False)

In [30]:
# #Read SQL and create a CSV

# # create sqlalchemy engine
# engine = create_engine(con)

# # read table from SQL database into a DataFrame
# poleworkshop_list_df = pd.read_sql_table('workshop_list', con=engine)

# # write DataFrame to CSV file
# poleworkshop_list_df.to_csv('workshop_list.csv', index=False)
